# Head Models and Coordinate Reference Systems (CRS)

In [ ]:
# This cells setups the environment when executed in Google Colab.
try:
    import google.colab
    !curl -s https://raw.githubusercontent.com/ibs-lab/cedalion/colab_setup/scripts/colab_setup.py -o colab_setup.py
    # Select branch with --branch "branch name" (default is "dev")
    %run colab_setup.py
except ImportError:
    pass

In [ ]:
import pyvista as pv
#pv.set_jupyter_backend('server')
pv.set_jupyter_backend('static')

import os
import xarray as xr

import cedalion
import cedalion.io
import cedalion.plots
import cedalion.datasets
import cedalion.imagereco.forward_model as fw

xr.set_options(display_expand_data=False);

## Loading the ICBM-152 head model

- the `TwoSurfaceHeadModel` holds the segmented MRT image and derived cortex and scalp surfaces
- we provide functionality to derive these surfaces from the masks or to load them from files

In [ ]:
# load pathes to segmentation data for the icbm-152 atlas
SEG_DATADIR, mask_files, landmarks_file = cedalion.datasets.get_icbm152_segmentation()

# create forward model class for icbm152 atlas
head_icbm152 = fw.TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR,
    mask_files=mask_files,
    brain_surface_file=os.path.join(SEG_DATADIR, "mask_brain.obj"),
    landmarks_ras_file=landmarks_file,
    brain_face_count=None,
    scalp_face_count=None,
)

## Visualization

In [ ]:
plt = pv.Plotter()
cedalion.plots.plot_surface(plt, head_icbm152.brain, color="#d3a6a1")
cedalion.plots.plot_surface(plt, head_icbm152.scalp, opacity=.1)
cedalion.plots.plot_labeled_points(plt, head_icbm152.landmarks, show_labels=True)
plt.show()

## Segmentation masks

The head model comprises masks for different tissue types: CSF, Gray Matter, White Matter, Scalp and Skull

In [ ]:
head_icbm152.segmentation_masks

## Coordinate System

- we need to distinguish several coordinate systems: voxel space, scanner space, subject space, ...
- geometric data types carry information about which crs they use
- transformations between coordinate systems through affine transformations


The head model is loaded in voxel space ('ijk')

In [ ]:
head_icbm152.crs

The head model contains initial landmarks ('Nz', 'Iz', 'LPA' and 'RPA') stored as a LabeledPointCloud.
The crs is stored as the name of the second dimension, easily retrievable through the `.points`-accessor 

In [ ]:
display(head_icbm152.landmarks)
display(head_icbm152.landmarks.points.crs)

Triangulated surface meshes of the scalp and brain:

In [ ]:
display(head_icbm152.brain)
display(head_icbm152.scalp)

In [ ]:
head_icbm152.t_ijk2ras # transformation from voxel to subject (RAS) space

Change to subject (RAS) space by applying an affine transformation on the head model.
This transforms all components.

Here, the subject space is called 'aligned' (the label is derived from information in the mask's nifti file)

The scanner space also uses physical units whereas coordinates in voxel space are dimensionless.

In [ ]:
trafo = head_icbm152.t_ijk2ras

head_icbm152_ras = head_icbm152.apply_transform(trafo)

display(head_icbm152_ras.crs)
display(head_icbm152_ras.landmarks.points.crs)
display(head_icbm152_ras.brain.crs)

display(head_icbm152.landmarks.pint.units)
display(head_icbm152_ras.landmarks.pint.units)